# Network flow modelling examples

In [1]:
import numpy as np
import pandas as pd
import gurobipy as gp
from gurobipy import GRB
import gurobipy_pandas as gppd

gppd.set_interactive()
gp.setParam('OutputFlag', 0)

## Min-cost flow network for max-flow computation

The following example sets up a min-cost flow network for computing maximum flow through a network. In general, network data should be stored as a multi-indexed dataframe, where:

- A row in the dataframe corresponds to an edge.
- The (from, to) pair in the index of a given row represents the direction of the arc.
- Data columns store edge attributes. In most cases this will be capacity and cost.

A basic assumption here is that the network is sparsely connected, and all edges are represented in the dataframe. So, if a (from, to) edge pair is not represented in `arc_data`, then the network does not contain an edge between this pair.

In [2]:
arc_data = pd.DataFrame([
    # Main components of the network: limited-capacity arcs
    {"from": 0, "to": 1, "capacity": 16, "cost": 0},
    {"from": 0, "to": 2, "capacity": 13, "cost": 0},
    {"from": 1, "to": 2, "capacity": 10, "cost": 0},
    {"from": 2, "to": 1, "capacity": 4, "cost": 0},
    {"from": 1, "to": 3, "capacity": 12, "cost": 0},
    {"from": 3, "to": 2, "capacity": 9, "cost": 0},
    {"from": 2, "to": 4, "capacity": 14, "cost": 0},
    {"from": 4, "to": 3, "capacity": 7, "cost": 0},
    {"from": 3, "to": 5, "capacity": 20, "cost": 0},
    {"from": 4, "to": 5, "capacity": 4, "cost": 0},
    # Add an uncapacitated edge from sink to source
    {"from": 5, "to": 0, "capacity": np.inf, "cost": -1},
]).set_index(["from", "to"])

arc_data

capacity  cost
from to                
0    1       16.0     0
     2       13.0     0
1    2       10.0     0
2    1        4.0     0
1    3       12.0     0
3    2        9.0     0
2    4       14.0     0
4    3        7.0     0
3    5       20.0     0
4    5        4.0     0
5    0        inf    -1

Min-cost network flow takes a simple and repeatable form. First, use the `.gppd.add_vars` dataframe accessor to add a flow variable for each arc in the network.

In [3]:
model = gp.Model("max-flow")
model.ModelSense = GRB.MINIMIZE

arc_df = arc_data.gppd.add_vars(
    model, ub="capacity", obj="cost", name="flow"
)
arc_df

capacity  cost                    flow
from to                                        
0    1       16.0     0  <gurobi.Var flow[0,1]>
     2       13.0     0  <gurobi.Var flow[0,2]>
1    2       10.0     0  <gurobi.Var flow[1,2]>
2    1        4.0     0  <gurobi.Var flow[2,1]>
1    3       12.0     0  <gurobi.Var flow[1,3]>
3    2        9.0     0  <gurobi.Var flow[3,2]>
2    4       14.0     0  <gurobi.Var flow[2,4]>
4    3        7.0     0  <gurobi.Var flow[4,3]>
3    5       20.0     0  <gurobi.Var flow[3,5]>
4    5        4.0     0  <gurobi.Var flow[4,5]>
5    0        inf    -1  <gurobi.Var flow[5,0]>

Using the `obj` attribute of the above method, and setting the model objective sense to minimization, sets up the linear objective function. In this case, the objective ultimately maximises the flow through the uncapacitated sink-source edge.

In [4]:
model.getObjective()

<gurobi.LinExpr: -1.0 flow[5,0]>

Flow constraints are simple to formulate using pandas `groupby` operations. Since the `arc_df` dataframe, containing arc attributes and flow variables, is indexed by from and to entries, we can group by each key to create linear expressions for inflow and outflow of each node in the network.

The inflow and outflow expressions are best grouped together in a single dataframe. This allows us to use the `.gppd.add_constrs` dataframe accessor to construct flow balance constraints.

In [5]:
constrs = (
    pd.DataFrame({
        "outflow": arc_df["flow"].groupby("from").sum(),
        "inflow": arc_df["flow"].groupby("to").sum(),
    })
    .gppd.add_constrs(model, "outflow == inflow", name="balance")
)
model.update()
constrs

,outflow,inflow,balance
0,"flow[0,1] + flow[0,2]","<gurobi.Var flow[5,0]>",<gurobi.Constr balance[0]>
1,"flow[1,2] + flow[1,3]","flow[0,1] + flow[2,1]",<gurobi.Constr balance[1]>
2,"flow[2,1] + flow[2,4]","flow[0,2] + flow[1,2] + flow[3,2]",<gurobi.Constr balance[2]>
3,"flow[3,2] + flow[3,5]","flow[1,3] + flow[4,3]",<gurobi.Constr balance[3]>
4,"flow[4,3] + flow[4,5]","<gurobi.Var flow[2,4]>",<gurobi.Constr balance[4]>
5,"<gurobi.Var flow[5,0]>","flow[3,5] + flow[4,5]",<gurobi.Constr balance[5]>


Do some quick debugging checks to show the constraints were constructed as expected:

In [6]:
constrs['balance'].apply(model.getRow)

0               flow[0,1] + flow[0,2] + -1.0 flow[5,0]
1    -1.0 flow[0,1] + flow[1,2] + -1.0 flow[2,1] + ...
2    -1.0 flow[0,2] + -1.0 flow[1,2] + flow[2,1] + ...
3    -1.0 flow[1,3] + flow[3,2] + -1.0 flow[4,3] + ...
4               -1.0 flow[2,4] + flow[4,3] + flow[4,5]
5          -1.0 flow[3,5] + -1.0 flow[4,5] + flow[5,0]
Name: balance, dtype: object

In [7]:
constrs['balance'].gppd.RHS

0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
5    0.0
Name: balance, dtype: float64

Solve the model, and extract the arc flows using the series accessors implemented in `gurobipy-pandas`:

In [8]:
model.optimize()
arc_df["flow"].gppd.X

from  to
0     1     16.0
      2      7.0
1     2      4.0
2     1      0.0
1     3     12.0
3     2      0.0
2     4     11.0
4     3      7.0
3     5     19.0
4     5      4.0
5     0     23.0
Name: flow, dtype: float64

It's also possible to use the constraints dataframe to compute total inflows and outflows from each node, based on the current solution, since it contains all the relevant linear expressions.

In [9]:
(
    constrs
    .assign(
        inflow_result=lambda df: df['inflow'].apply(gp.LinExpr).gppd.get_value(),
        outflow_result=lambda df: df['outflow'].apply(gp.LinExpr).gppd.get_value(),
    )
)

,outflow,inflow,balance,inflow_result,outflow_result
0,"flow[0,1] + flow[0,2]","<gurobi.Var flow[5,0] (value 23.0)>",<gurobi.Constr balance[0]>,23.0,23.0
1,"flow[1,2] + flow[1,3]","flow[0,1] + flow[2,1]",<gurobi.Constr balance[1]>,16.0,16.0
2,"flow[2,1] + flow[2,4]","flow[0,2] + flow[1,2] + flow[3,2]",<gurobi.Constr balance[2]>,11.0,11.0
3,"flow[3,2] + flow[3,5]","flow[1,3] + flow[4,3]",<gurobi.Constr balance[3]>,19.0,19.0
4,"flow[4,3] + flow[4,5]","<gurobi.Var flow[2,4] (value 11.0)>",<gurobi.Constr balance[4]>,11.0,11.0
5,"<gurobi.Var flow[5,0] (value 23.0)>","flow[3,5] + flow[4,5]",<gurobi.Constr balance[5]>,23.0,23.0


## Networks with demand nodes

Another model we might want to tackle is a network consisting of different node types:

- transshipment nodes with no demand or supply
- supply nodes with an external input
- demand nodes with an external output

The input dataframe structure for the network is the same here. Again, the model is structured such that if an edge is not in the network, it should not appear in this datase (and hence, cannot take any flows).

In [10]:
arc_data = pd.DataFrame([
    {"from": 0, "to": 1, "capacity": 16, "cost": 1},
    {"from": 0, "to": 2, "capacity": 13, "cost": 2},
    {"from": 1, "to": 2, "capacity": 10, "cost": 1},
    {"from": 2, "to": 1, "capacity": 4, "cost": 3},
    {"from": 1, "to": 3, "capacity": 12, "cost": 1},
    {"from": 3, "to": 2, "capacity": 9, "cost": 2},
    {"from": 2, "to": 4, "capacity": 14, "cost": 3},
    {"from": 4, "to": 3, "capacity": 7, "cost": 1},
    {"from": 3, "to": 5, "capacity": 20, "cost": 2},
    {"from": 4, "to": 5, "capacity": 4, "cost": 1},
]).set_index(["from", "to"])

arc_data

capacity  cost
from to                
0    1         16     1
     2         13     2
1    2         10     1
2    1          4     3
1    3         12     1
3    2          9     2
2    4         14     3
4    3          7     1
3    5         20     2
4    5          4     1

We need to also store the supply/demand requirements for each node. Here, we store this data in a dataframe, indexed by node, with a single demand column. A negative demand entry indicates a supply node, positive a demand. Transshipment nodes do not appear in this frame (implicitly supply, & demand are zero).

In [11]:
demand_data = pd.DataFrame([
    {"node": 0, "demand": -23},
    {"node": 5, "demand": 23}
]).set_index("node")
demand_data

,demand
node,
0,-23
5,23


Add flow variables using the same approach as before (one per arc in the dataframe):

In [12]:
model = gp.Model("supply-demand")
model.ModelSense = GRB.MINIMIZE

arc_df = arc_data.gppd.add_vars(model, name="flow", ub="capacity", obj="cost")
arc_df

capacity  cost                    flow
from to                                        
0    1         16     1  <gurobi.Var flow[0,1]>
     2         13     2  <gurobi.Var flow[0,2]>
1    2         10     1  <gurobi.Var flow[1,2]>
2    1          4     3  <gurobi.Var flow[2,1]>
1    3         12     1  <gurobi.Var flow[1,3]>
3    2          9     2  <gurobi.Var flow[3,2]>
2    4         14     3  <gurobi.Var flow[2,4]>
4    3          7     1  <gurobi.Var flow[4,3]>
3    5         20     2  <gurobi.Var flow[3,5]>
4    5          4     1  <gurobi.Var flow[4,5]>

In [13]:
model.getObjective()

<gurobi.LinExpr: flow[0,1] + 2.0 flow[0,2] + flow[1,2] + 3.0 flow[2,1] + flow[1,3] + 2.0 flow[3,2] + 3.0 flow[2,4] + flow[4,3] + 2.0 flow[3,5] + flow[4,5]>

Construction of flow balance is similar to the max-flow example, but we also need to include the supply/demand data.

One important note is that since various nodes are missing either inflow, outflow, or demand based on the network structure, there will be missing values in this dataframe. `gurobipy-pandas` explicitly disallows this, so we need to fill in the data.

Based on the way we structured the data initially: a missing entry in the demand column indicates a transhipment node (zero supply or demand). Similarly, if a node has no inflow edge, or no outflow edge, we can safely say it's total inflow or outflow, respectively, are zero. So applying `.fillna(0)` to this dataframe gives us the correct model.

In [14]:
pd.DataFrame({
    "inflow": arc_df["flow"].groupby("to").sum(),
    "outflow": arc_df["flow"].groupby("from").sum(),
    "demand": demand_data["demand"],
})

,inflow,outflow,demand
0,NaN,"flow[0,1] + flow[0,2]",-23.0
1,"flow[0,1] + flow[2,1]","flow[1,2] + flow[1,3]",NaN
2,"flow[0,2] + flow[1,2] + flow[3,2]","flow[2,1] + flow[2,4]",NaN
3,"flow[1,3] + flow[4,3]","flow[3,2] + flow[3,5]",NaN
4,"<gurobi.Var flow[2,4]>","flow[4,3] + flow[4,5]",NaN
5,"flow[3,5] + flow[4,5]",NaN,23.0


Applying `fillna`, then using method chaining to construct constraints, we get the following result:

In [15]:
balance_df = (
    pd.DataFrame({
        "inflow": arc_df["flow"].groupby("to").sum(),
        "outflow": arc_df["flow"].groupby("from").sum(),
        "demand": demand_data["demand"],
    })
    .fillna(0)   # zero fill (some nodes have no in, out, or demand)
    .gppd.add_constrs(model, "inflow - outflow == demand", name="balance")
)
balance_df

,inflow,outflow,demand,balance
0,0,"flow[0,1] + flow[0,2]",-23.0,<gurobi.Constr balance[0]>
1,"flow[0,1] + flow[2,1]","flow[1,2] + flow[1,3]",0.0,<gurobi.Constr balance[1]>
2,"flow[0,2] + flow[1,2] + flow[3,2]","flow[2,1] + flow[2,4]",0.0,<gurobi.Constr balance[2]>
3,"flow[1,3] + flow[4,3]","flow[3,2] + flow[3,5]",0.0,<gurobi.Constr balance[3]>
4,"<gurobi.Var flow[2,4]>","flow[4,3] + flow[4,5]",0.0,<gurobi.Constr balance[4]>
5,"flow[3,5] + flow[4,5]",0,23.0,<gurobi.Constr balance[5]>


We can again inspect the constraints they were built to confirm:

In [16]:
balance_df['balance'].apply(model.getRow)

0                      -1.0 flow[0,1] + -1.0 flow[0,2]
1    flow[0,1] + -1.0 flow[1,2] + flow[2,1] + -1.0 ...
2    flow[0,2] + flow[1,2] + -1.0 flow[2,1] + flow[...
3    flow[1,3] + -1.0 flow[3,2] + flow[4,3] + -1.0 ...
4          flow[2,4] + -1.0 flow[4,3] + -1.0 flow[4,5]
5                                flow[3,5] + flow[4,5]
Name: balance, dtype: object

In [17]:
balance_df['balance'].gppd.RHS

0   -23.0
1     0.0
2     0.0
3     0.0
4     0.0
5    23.0
Name: balance, dtype: float64

Solve the model, and append the results as a column in the flows dataframe. We can easily inspect this to see that capacities are respected:

In [18]:
model.optimize()
arc_df.assign(result=lambda df: df['flow'].gppd.X)

capacity  cost                                 flow  result
from to                                                             
0    1         16     1  <gurobi.Var flow[0,1] (value 12.0)>    12.0
     2         13     2  <gurobi.Var flow[0,2] (value 11.0)>    11.0
1    2         10     1   <gurobi.Var flow[1,2] (value 0.0)>     0.0
2    1          4     3   <gurobi.Var flow[2,1] (value 0.0)>     0.0
1    3         12     1  <gurobi.Var flow[1,3] (value 12.0)>    12.0
3    2          9     2   <gurobi.Var flow[3,2] (value 0.0)>     0.0
2    4         14     3  <gurobi.Var flow[2,4] (value 11.0)>    11.0
4    3          7     1   <gurobi.Var flow[4,3] (value 7.0)>     7.0
3    5         20     2  <gurobi.Var flow[3,5] (value 19.0)>    19.0
4    5          4     1   <gurobi.Var flow[4,5] (value 4.0)>     4.0